## Menu Recommendation

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

### Import Data

In [3]:
cd D:\[졸작] 캡스톤디자인(2)\recommendation

D:\[졸작] 캡스톤디자인(2)\recommendation


In [4]:
data = pd.read_csv("../data collection/data/menu_db.csv", sep=',', encoding='cp949')
print(data.head())

   id  restaurant_id category      menu  price restaurant_name
0   1              1       국밥     양평해장국   9000    일품양평해장국 인천대점
1   2              1       국밥  양평해장국(특)  10000    일품양평해장국 인천대점
2   3              1       국밥    뼈다귀해장국  10000    일품양평해장국 인천대점
3   4              1       국밥     얼큰순대국   9000    일품양평해장국 인천대점
4   5              1       국밥       내장탕  13000    일품양평해장국 인천대점


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               339 non-null    int64 
 1   restaurant_id    339 non-null    int64 
 2   category         339 non-null    object
 3   menu             339 non-null    object
 4   price            339 non-null    int64 
 5   restaurant_name  339 non-null    object
dtypes: int64(3), object(3)
memory usage: 16.0+ KB


In [6]:
data = data[['category', 'menu', 'restaurant_name']]
print(data.head())

  category      menu restaurant_name
0       국밥     양평해장국    일품양평해장국 인천대점
1       국밥  양평해장국(특)    일품양평해장국 인천대점
2       국밥    뼈다귀해장국    일품양평해장국 인천대점
3       국밥     얼큰순대국    일품양평해장국 인천대점
4       국밥       내장탕    일품양평해장국 인천대점


In [7]:
print(data.isnull().sum())

category           0
menu               0
restaurant_name    0
dtype: int64


In [8]:
data = data.dropna()

### Based On 'Category'

In [9]:
feature = data['category'].tolist()
rest = text.TfidfVectorizer(input=feature)
rest_matrix = rest.fit_transform(feature)
similarity = cosine_similarity(rest_matrix)

In [11]:
rest

TfidfVectorizer(input=['국밥', '국밥', '국밥', '국밥', '국밥', '국밥', '고기', '고기', '고기',
                       '고기', '사이드', '일식', '일식', '일식', '일식', '일식', '일식', '일식',
                       '일식', '일식', '샐러드', '샐러드', '샐러드', '샐러드', '샐러드', '샌드위치',
                       '샌드위치', '일식', '일식', '일식', ...])

In [12]:
rest_matrix

<339x20 sparse matrix of type '<class 'numpy.float64'>'
	with 335 stored elements in Compressed Sparse Row format>

In [13]:
similarity

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [15]:
indices = pd.Series(data.index, index=data['menu']).drop_duplicates() # 중복 행 제거

In [16]:
indices

menu
양평해장국         0
양평해장국(특)      1
뼈다귀해장국        2
얼큰순대국         3
내장탕           4
           ... 
삼겹살덮밥       334
육회비빔밥       335
순두부국밥       336
우동국밥        337
수육곰탕        338
Length: 339, dtype: int64

In [17]:
def menu_recommendation(name, similarity = similarity):
    index = indices[name]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    restaurantindices = [i[0] for i in similarity_scores]
    return data['menu'].iloc[restaurantindices]

In [18]:
print(menu_recommendation("양평해장국"))

0         양평해장국
1      양평해장국(특)
2        뼈다귀해장국
3         얼큰순대국
4           내장탕
5           육개장
237        순대국밥
238        수육국밥
239        얼큰국밥
336       순두부국밥
Name: menu, dtype: object


In [19]:
print(menu_recommendation("김치만두"))

41       칼만두굿
45       떡만두국
47     감자고기만두
48       김치만두
49       갈비만두
253      교자만두
254      갈비만두
255      튀김만두
333       물만두
0       양평해장국
Name: menu, dtype: object
